# Система анонимизации текста на русском языке

Этот ноутбук содержит полную систему анонимизации персональных данных в текстах на русском языке. Система использует:
- Natasha для распознавания именованных сущностей (NER)
- Регулярные выражения для структурированных данных
- Faker для генерации заменяющих данных
- pymorphy3 для сохранения морфологии русского языка

## Возможности системы:
- Распознавание и анонимизация имен, фамилий
- Анонимизация email, телефонов, паспортов, ИНН
- Анонимизация банковских карт, ОГРНИП, IP-адресов
- Два режима работы: маскирование и замена
- Оценка качества анонимизации с помощью BERTScore

## Установка зависимостей

Сначала установим все необходимые библиотеки:

In [ ]:
!pip install natasha>=1.6.0 faker>=39.0.0 bert-score>=0.3.13 pymorphy3>=2.0.6 transformers>=4.30.0 pandas tqdm

## Импорт библиотек

In [ ]:
import os
import re
import random
import json
import numpy as np
from collections import defaultdict
from typing import List, Tuple, Union, Dict, Any

# NLP библиотеки
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)
from faker import Faker
import pymorphy3

# Для оценки
from bert_score import score
import pandas as pd
from tqdm import tqdm

## Классы для распознавания именованных сущностей (NER)

In [ ]:
class NatashaNER:
    def __init__(self):
        self.segmenter = Segmenter()
        self.morph_vocab = MorphVocab()
        self.emb = NewsEmbedding()
        self.morph_tagger = NewsMorphTagger(self.emb)
        self.syntax_parser = NewsSyntaxParser(self.emb)
        self.ner_tagger = NewsNERTagger(self.emb)
        self.names_extractor = NamesExtractor(self.morph_vocab)

    def extract_names(self, text):
        doc = Doc(text)
        
        doc.segment(self.segmenter)
        doc.tag_morph(self.morph_tagger)
        doc.parse_syntax(self.syntax_parser)
        doc.tag_ner(self.ner_tagger)
        
        result = []
        
        if doc.spans is not None:
            for span in doc.spans:
                if span.type == PER:
                    span.normalize(self.morph_vocab)
                    span.extract_fact(self.names_extractor)
                    
                    # Zip tokens and slots
                    if span.fact and span.fact.slots:
                        for token, slot in zip(span.tokens, span.fact.slots):
                            # Determine tag based on slot key
                            if slot.key == 'last':
                                tag = 'SURNAME'
                            elif slot.key == 'first':
                                tag = 'NAME'
                            else:
                                continue  # Skip other slots
                            
                            # Get morphological features
                            feats = token.feats
                            
                            # Get start and end positions
                            start = token.start
                            end = token.stop
                            
                            result.append([tag, feats, start, end])
        
        return result

In [ ]:
class RegexNER:
    def __init__(self):
        # Регулярное выражение для номера банковской карты (Visa, Mastercard, Maestro, Мир)
        # Формат: 13-19 цифр, с проверкой по алгоритму Луна
        self.card_pattern = re.compile(r'\b(?:\d[ -]*?){13,19}\b')
        
        # Регулярное выражение для ОГРНИП (15 цифр)
        # Формат: XXXXXXXXXXXXXXX, где последняя цифра - контрольное число (N14 mod 11 mod 10)
        self.ogrnip_pattern = re.compile(r'\b\d{15}\b')
        
        # Регулярное выражение для IP-адреса
        self.ip_pattern = re.compile(r'\b(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\b')
        
        # Регулярное выражение для email
        self.email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b')
        
        # Регулярное выражение для номера свидетельства о рождении
        # Формат: VII-АБ №123456 (римские цифры, русские буквы, номер)
        self.birth_cert_pattern = re.compile(r'\b(?:[IVXLCDM]+-[А-ЯЁ]{2}\s*№\d{6})\b', re.IGNORECASE)
        
        # Регулярное выражение для кода КЛАДР
        # Формат: XX YY ZZZ QQ, где XX - код региона, YY - код района, ZZZ - код города/населенного пункта, QQ - код улицы
        self.kladr_pattern = re.compile(r'\b\d{2}\s*\d{2}\s*\d{3}\s*\d{2}\b')
    
    def luhn_algorithm(self, card_number):
        """Проверка номера карты по алгоритму Луна"""
        card_number = card_number.replace(' ', '').replace('-', '')
        if not card_number.isdigit():
            return False
        
        total = 0
        for i, digit in enumerate(card_number):
            digit = int(digit)
            if i % 2 == len(card_number) % 2:
                digit *= 2
                if digit > 9:
                    digit -= 9
            total += digit
        
        return total % 10 == 0
    
    def validate_ogrnip(self, ogrnip):
        """Проверка ОГРНИП"""
        if len(ogrnip) != 15 or not ogrnip.isdigit():
            return False
        
        # Последняя цифра - контрольное число
        main_part = int(ogrnip[:14])
        control_digit = int(ogrnip[14])
        
        # Вычисляем контрольное число
        calculated_control = main_part % 11 % 10
        
        return calculated_control == control_digit
    
    def extract_entities(self, text):
        result = []
        
        # Поиск номеров банковских карт
        for match in self.card_pattern.finditer(text):
            card_number = match.group().replace(' ', '').replace('-', '')
            if self.luhn_algorithm(card_number):
                result.append(['CARD_NUMBER', 'valid_card', match.start(), match.end()])
            else:
                result.append(['CARD_NUMBER', 'invalid_card', match.start(), match.end()])
        
        # Поиск ОГРНИП
        for match in self.ogrnip_pattern.finditer(text):
            if self.validate_ogrnip(match.group()):
                result.append(['OGRNIP', 'valid_ogrnip', match.start(), match.end()])
        
        # Поиск IP-адресов
        for match in self.ip_pattern.finditer(text):
            result.append(['IP_ADDRESS', 'valid_ip', match.start(), match.end()])
        
        # Поиск email
        for match in self.email_pattern.finditer(text):
            result.append(['EMAIL', 'valid_email', match.start(), match.end()])
        
        # Поиск номеров свидетельств о рождении
        for match in self.birth_cert_pattern.finditer(text):
            result.append(['BIRTH_CERT', 'valid_birth_cert', match.start(), match.end()])
        
        # Поиск кодов КЛАДР
        for match in self.kladr_pattern.finditer(text):
            result.append(['KLADR', 'valid_kladr', match.start(), match.end()])
        
        return result

## Класс для анонимизации персональных данных

In [ ]:
class PDAnonymizer:
    def __init__(self):
        self.faker = Faker('ru_RU')
        self.morph = pymorphy3.MorphAnalyzer()
    
    def _get_morph_features(self, word: str) -> Dict[str, Any]:
        """Получить морфологические характеристики слова"""
        if not isinstance(word, str) or not word.strip():
            return {'gender': 'masc', 'number': 'sing', 'case': 'nomn'}
            
        try:
            parsed = self.morph.parse(word)[0]
            return {
                'gender': parsed.tag.gender if parsed.tag.gender else 'masc',
                'number': parsed.tag.number if parsed.tag.number else 'sing',
                'case': parsed.tag.case if parsed.tag.case else 'nomn'
            }
        except Exception:
            return {'gender': 'masc', 'number': 'sing', 'case': 'nomn'}
    
    def _inflect_word(self, word: str, target_features: Dict[str, Any]) -> str:
        """Просклонять слово по заданным характеристикам"""
        if not isinstance(word, str) or not word.strip():
            return word
            
        try:
            parsed = self.morph.parse(word)[0]
            
            # Создаем набор тегов для склонения
            target_set = set()
            if target_features.get('gender'):
                target_set.add(target_features['gender'])
            if target_features.get('number'):
                target_set.add(target_features['number'])
            if target_features.get('case'):
                target_set.add(target_features['case'])
            
            if target_set:
                inflected = parsed.inflect(target_set)
                if inflected:
                    return inflected.word
                else:
                    return word
            else:
                return word
                
        except Exception:
            return word
    
    def anonymize_name(self, name: str, features: dict | None = None) -> str:
        """Обезличивание имени с сохранением морфологии"""
        try:
            # Проверка на корректность данных
            if not isinstance(name, str) or not name.strip():
                return "Некорректно введены данные"
            
            # Проверка на цифры и специальные символы в имени
            if re.search(r'[\d@#$%^&*()_+=\[\]{}|;:",.<>?/\\]', name):
                return "Некорректно введены данные"
            
            # Получаем морфологические характеристики оригинала
            original_features = features or self._get_morph_features(name)
            
            # Генерируем случайное имя того же рода с помощью Faker
            if original_features.get('gender') == 'femn':
                new_name = self.faker.first_name_female()
            else:
                new_name = self.faker.first_name_male()
            
            # Склоняем новое имя по характеристикам оригинала
            anonymized_name = self._inflect_word(new_name, original_features)
            
            # Возвращаем результат
            return anonymized_name.capitalize()
            
        except Exception:
            return "Некорректно введены данные"
    
    def anonymize_last_name(self, last_name: str, features: dict | None = None) -> str:
        """Обезличивание фамилии с сохранением морфологии"""
        try:
            # Проверка на корректность данных
            if not isinstance(last_name, str) or not last_name.strip():
                return "Некорректно введены данные"
            
            # Проверка на цифры и специальные символы в фамилии
            if re.search(r'[\d@#$%^&*()_+=\[\]{}|;:",.<>?/\\]', last_name):
                return "Некорректно введены данные"
            
            # Получаем морфологические характеристики оригинала
            original_features = features or self._get_morph_features(last_name)
            
            # Генерируем случайную фамилию того же рода с помощью Faker
            if original_features.get('gender') == 'femn':
                new_last_name = self.faker.last_name_female()
            else:
                new_last_name = self.faker.last_name_male()
            
            # Склоняем новую фамилию по характеристикам оригинала
            anonymized_last_name = self._inflect_word(new_last_name, original_features)
            
            # Возвращаем результат
            return anonymized_last_name.capitalize()
            
        except Exception:
            return "Некорректно введены данные"
    
    def anonymize_email(self, email: str) -> str:
        """Обезличивание email с сохранением домена"""
        try:
            # Проверка на корректность данных
            if not isinstance(email, str) or not email.strip():
                return "Некорректно введены данные"
            
            # Проверка формата email
            if '@' not in email:
                return "Некорректно введены данные"
            
            local_part, domain = email.split('@', 1)
            new_local = self.faker.user_name()
            return f"{new_local}@{domain}"
            
        except Exception:
            return "Некорректно введены данные"
    
    def _calc_inn_control(self, digits: str, weights: list[int]) -> str:
        s = sum(int(d) * w for d, w in zip(digits, weights))
        return str(s % 11 % 10)

    def anonymize_inn(self, inn: str) -> str:
        """Обезличивание ИНН с сохранением первых 4 цифр и контрольного разряда"""
        try:
            if not isinstance(inn, str) or not inn.isdigit():
                return "Некорректно введены данные"

            if len(inn) == 10:
                base = ''.join(str(random.randint(0, 9)) for _ in range(9))
                control = self._calc_inn_control(
                    base, [2, 4, 10, 3, 5, 9, 4, 6, 8]
                )
                return base + control

            if len(inn) == 12:
                base = ''.join(str(random.randint(0, 9)) for _ in range(10))
                c1 = self._calc_inn_control(
                    base[:10], [7, 2, 4, 10, 3, 5, 9, 4, 6, 8]
                )
                c2 = self._calc_inn_control(
                    base[:11] + c1,
                    [3, 7, 2, 4, 10, 3, 5, 9, 4, 6, 8]
                )
                return base[:10] + c1 + c2

            return "Некорректно введены данные"
        except Exception:
            return "Некорректно введены данные"
    
    def anonymize_phone(self, phone: str) -> str:
        """Обезличивание номера телефона с сохранением формата"""
        try:
            # Проверка на корректность данных
            if not isinstance(phone, str) or not phone.strip():
                return "Некорректно введены данные"
            
            # Строгая проверка: допустимы только цифры, +, (), - и пробелы
            if not re.match(r'^[\d\+\-\(\)\s]+$', phone):
                return "Некорректно введены данные"
            
            digits = re.findall(r'\d', phone)
            
            # Проверка минимальной длины телефона
            if len(digits) < 7:
                return "Некорректно введены данные"
                
            preserved_digits = digits[:4]
            new_digits = [str(random.randint(0, 9)) for _ in range(len(digits) - 4)]
            
            result = []
            digit_index = 0
            new_all_digits = preserved_digits + new_digits
            
            for char in phone:
                if char.isdigit():
                    if digit_index < len(new_all_digits):
                        result.append(new_all_digits[digit_index])
                        digit_index += 1
                    else:
                        result.append(char)
                else:
                    result.append(char)
            
            return ''.join(result)
            
        except Exception:
            return "Некорректно введены данные"
    
    def anonymize_passport(self, passport: str) -> str:
        try:
            m = re.fullmatch(r'(\d{2})\s?(\d{2})\s?(\d{6})', passport)
            if not m:
                return "Некорректно введены данные"

            series = ''.join(str(random.randint(0, 9)) for _ in range(4))
            number = ''.join(str(random.randint(0, 9)) for _ in range(6))
            return f"{series[:2]} {series[2:]} {number}"
        except Exception:
            return "Некорректно введены данные"
    
    def _luhn_checksum(self, digits: str) -> str:
        s = 0
        for i, d in enumerate(reversed(digits)):
            n = int(d)
            if i % 2 == 0:
                n *= 2
                if n > 9:
                    n -= 9
            s += n
        return str((10 - s % 10) % 10)
    
    def anonymize_card(self, card: str) -> str:
        try:
            digits = re.sub(r'\D', '', card)
            if not (13 <= len(digits) <= 19):
                return "Некорректно введены данные"

            body = ''.join(str(random.randint(0, 9)) for _ in range(len(digits) - 1))
            check = self._luhn_checksum(body)
            new_digits = body + check

            result = []
            i = 0
            for ch in card:
                if ch.isdigit():
                    result.append(new_digits[i])
                    i += 1
                else:
                    result.append(ch)
            return ''.join(result)
        except Exception:
            return "Некорректно введены данные"
    
    def anonymize_ogrnip(self, ogrnip: str) -> str:
        try:
            if not ogrnip.isdigit() or len(ogrnip) != 15:
                return "Некорректно введены данные"

            base = ''.join(str(random.randint(0, 9)) for _ in range(14))
            control = str(int(base) % 13 % 10)
            return base + control
        except Exception:
            return "Некорректно введены данные"
    

    def anonymize_ip(self, ip: str) -> str:
        try:
            parts = ip.split('.')
            if len(parts) != 4:
                return "Некорректно введены данные"

            new_parts = [str(random.randint(1, 254)) for _ in range(4)]
            return '.'.join(new_parts)
        except Exception:
            return "Некорректно введены данные"
 
    def anonymize_birth_cert(self, cert: str) -> str:
        try:
            # Проверяем формат с римскими цифрами
            m = re.fullmatch(r'([IVXLCDM]+)-([А-ЯЁ]{2})\s*№(\d{6})', cert, re.IGNORECASE)
            if not m:
                return "Некорректно введены данные"

            # Генерируем новые римские цифры (от 1 до 20)
            roman_numeral = self._int_to_roman(random.randint(1, 20))
            letters = ''.join(random.choice('АБВГДЕЖЗИКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ') for _ in range(2))
            number = ''.join(str(random.randint(0, 9)) for _ in range(6))
            return f"{roman_numeral}-{letters} №{number}"
        except Exception:
            return "Некорректно введены данные"
    
    def _int_to_roman(self, num: int) -> str:
        """Преобразование целого числа в римские цифры"""
        val = [
            1000, 900, 500, 400,
            100, 90, 50, 40,
            10, 9, 5, 4,
            1
        ]
        syb = [
            "M", "CM", "D", "CD",
            "C", "XC", "L", "XL",
            "X", "IX", "V", "IV",
            "I"
        ]
        roman_num = ''
        i = 0
        while num > 0:
            for _ in range(num // val[i]):
                roman_num += syb[i]
                num -= val[i]
            i += 1
        return roman_num
    
    def anonymize_kladr(self, kladr: str) -> str:
        try:
            # Удаляем все пробелы и проверяем формат
            digits = re.sub(r'\s', '', kladr)
            if not digits.isdigit() or len(digits) < 9:
                return "Некорректно введены данные"

            # Генерируем новый код КЛАДР той же длины
            new_digits = ''.join(str(random.randint(0, 9)) for _ in range(len(digits)))
            
            # Восстанавливаем исходный формат (с пробелами)
            result = new_digits
            if ' ' in kladr:
                # Восстанавливаем пробелы на тех же позициях
                for i, char in enumerate(kladr):
                    if char == ' ' and i < len(result):
                        result = result[:i] + ' ' + result[i:]
            
            return result
        except Exception:
            return "Некорректно введены данные"

## Основной класс анонимизатора

In [ ]:
class NERAnonymizer:
    def __init__(self, mode: str = "mask"):
        """
        Инициализация анонимизатора
        
        Args:
            mode: Режим работы - "mask" для маскирования или "replace" для замены
        """
        if mode not in ["mask", "replace"]:
            raise ValueError("Режим должен быть 'mask' или 'replace'")
            
        self.mode = mode
        self.natasha_ner = NatashaNER()
        self.regex_ner = RegexNER()
        self.pd_anonymizer = PDAnonymizer()
    
    def anonymize(self, text: str) -> str:
        """
        Анонимизация текста с использованием NER
        
        Args:
            text: Исходный текст
            
        Returns:
            Анонимизированный текст
        """
        # Получаем все сущности из текста
        natasha_entities = self.natasha_ner.extract_names(text)
        regex_entities = self.regex_ner.extract_entities(text)
        
        # Объединяем все сущности и сортируем по начальной позиции
        all_entities = natasha_entities + regex_entities
        all_entities.sort(key=lambda x: x[2])  # Сортировка по start
        
        # Создаем список замен для применения
        replacements = []
        
        for entity in all_entities:
            tag, feats, start, end = entity
            original_text = text[start:end]
            
            if self.mode == "mask":
                # Режим маскирования
                replacement = "#" * len(original_text)
            else:
                # Режим замены
                replacement = self._get_replacement(tag, original_text, feats)
            
            replacements.append((start, end, replacement))
        
        # Применяем замены в обратном порядке, чтобы не сбить индексы
        result_text = text
        for start, end, replacement in sorted(replacements, key=lambda x: x[0], reverse=True):
            result_text = result_text[:start] + replacement + result_text[end:]
        
        return result_text
    
    def _get_replacement(self, tag: str, original_text: str, feats: str) -> str:
        """
        Получение замены для сущности в режиме replace
        
        Args:
            tag: Тип сущности
            original_text: Оригинальный текст
            feats: Морфологические признаки
            
        Returns:
            Замененный текст
        """
        if tag == 'NAME':
            return self.pd_anonymizer.anonymize_name(original_text, features=None)
        elif tag == 'SURNAME':
            return self.pd_anonymizer.anonymize_last_name(original_text, features=None)
        elif tag == 'EMAIL':
            return self.pd_anonymizer.anonymize_email(original_text)
        elif tag == 'CARD_NUMBER':
            return self.pd_anonymizer.anonymize_card(original_text)
        elif tag == 'OGRNIP':
            return self.pd_anonymizer.anonymize_ogrnip(original_text)
        elif tag == 'IP_ADDRESS':
            return self.pd_anonymizer.anonymize_ip(original_text)
        elif tag == 'BIRTH_CERT':
            return self.pd_anonymizer.anonymize_birth_cert(original_text)
        elif tag == 'KLADR':
            return self.pd_anonymizer.anonymize_kladr(original_text)
        else:
            # Для неизвестных типов сущностей используем маскирование
            return "#" * len(original_text)
    
    def extract_and_anonymize(self, text: str) -> Tuple[str, List[List[Union[str, int]]]]:
        """
        Извлечение сущностей и анонимизация текста
        
        Args:
            text: Исходный текст
            
        Returns:
            Кортеж из анонимизированного текста и списка найденных сущностей
        """
        # Получаем все сущности
        natasha_entities = self.natasha_ner.extract_names(text)
        regex_entities = self.regex_ner.extract_entities(text)
        
        # Объединяем и сортируем сущности
        all_entities = natasha_entities + regex_entities
        all_entities.sort(key=lambda x: x[2])  # Сортировка по start
        
        # Анонимизируем текст
        anonymized_text = self.anonymize(text)
        
        return anonymized_text, all_entities

## Демонстрация работы системы

Теперь протестируем нашу систему анонимизации на примере текста с различными типами персональных данных.

In [ ]:
# Создаем анонимизаторы в двух режимах
mask_anonymizer = NERAnonymizer(mode="mask")
replace_anonymizer = NERAnonymizer(mode="replace")

# Тестовый текст с различными типами персональных данных
test_text = """
Регистрация ИП: Виноградов Никита Олегович, ОГРНИП 255547853460739, ИНН 7801920588.
Адрес по КЛАДР: 27 328 191 574. Email для связи: никита764@list.ru.
Зарплатный проект: сотрудник Виноградов Никита Олегович, ИНН 7801920588, карта 1558 5701 8528 1280.
Код КЛАДР для налоговой: 27 328 191 574. Клиент Виноградов Никита Олегович предоставил документы:
паспорт 3255 141253, ИНН 7801920588, свидетельство о рождении X-VI №901709.
Регистрация в системе: пользователь Виноградов Никита Олегович, email никита764@list.ru,
ИНН 7801920588, ОГРНИП 255547853460739 (если ИП). Для верификации необходимы данные:
Виноградов Никита Олегович, ИНН 7801920588, паспорт 3255 141253, email никита764@list.ru,
IP-адрес 76.79.127.227. Данные предпринимателя: Виноградов Никита Олегович,
ОГРНИП 255547853460739. Банковская карта 1558 5701 8528 1280. ИНН 7801920588.
Адрес по КЛАДР: 27 328 191 574.
"""

print("Исходный текст:")
print(test_text)

# Анонимизация в режиме маскирования
masked_text, entities = mask_anonymizer.extract_and_anonymize(test_text)
print("\nРежим маскирования:")
print(masked_text)
print(f"\nНайденные сущности: {len(entities)}")

# Анонимизация в режиме замены
replaced_text, entities = replace_anonymizer.extract_and_anonymize(test_text)
print("\n\nРежим замены:")
print(replaced_text)
print(f"\nНайденные сущности: {len(entities)}")

## Интерактивная демонстрация

Вы можете ввести свой текст для анонимизации:

In [ ]:
# Интерактивная ячейка для ввода текста
# В Google Colab используйте input(), в Jupyter можно использовать виджеты

try:
    # Для Jupyter Notebook с виджетами
    import ipywidgets as widgets
    from IPython.display import display
    
    # Создаем виджет для ввода текста
    text_input = widgets.Textarea(
        value='Введите текст для анонимизации...',
        placeholder='Введите текст для анонимизации...',
        description='Текст:',
        disabled=False,
        layout=widgets.Layout(width='100%', height='150px')
    )
    
    # Кнопка для анонимизации
    anonymize_button = widgets.Button(description='Анонимизировать')
    
    # Область для вывода результатов
    output = widgets.Output()
    
    def on_anonymize_button_clicked(b):
        with output:
            output.clear_output()
            text = text_input.value
            
            # Анонимизация в режиме маскирования
            masked_text, mask_entities = mask_anonymizer.extract_and_anonymize(text)
            print("Режим маскирования:")
            print(masked_text)
            print(f"Найдено сущностей: {len(mask_entities)}")
            
            # Анонимизация в режиме замены
            replaced_text, replace_entities = replace_anonymizer.extract_and_anonymize(text)
            print("\nРежим замены:")
            print(replaced_text)
            print(f"Найдено сущностей: {len(replace_entities)}")
    
    anonymize_button.on_click(on_anonymize_button_clicked)
    
    # Отображаем виджеты
    display(text_input, anonymize_button, output)
    
except ImportError:
    # Если ipywidgets недоступен, используем обычный input
    print("Интерактивный режим (введите текст и нажмите Enter):")
    text = input("Введите текст для анонимизации: ")
    
    # Анонимизация в режиме маскирования
    masked_text, mask_entities = mask_anonymizer.extract_and_anonymize(text)
    print("\nРежим маскирования:")
    print(masked_text)
    print(f"Найдено сущностей: {len(mask_entities)}")
    
    # Анонимизация в режиме замены
    replaced_text, replace_entities = replace_anonymizer.extract_and_anonymize(text)
    print("\nРежим замены:")
    print(replaced_text)
    print(f"Найдено сущностей: {len(replace_entities)}")

## Оценка качества анонимизации

Теперь оценим качество нашей системы анонимизации с использованием BERTScore. Эта метрика позволяет измерить семантическое сходство между исходным и анонимизированным текстом.

In [ ]:
def calculate_bert_score(original_texts, anonymized_texts, model_type="bert-base-multilingual-cased"):
    """
    Calculate BERTScore between original and anonymized texts
    
    Args:
        original_texts: List of original texts
        anonymized_texts: List of anonymized texts
        model_type: BERT model to use for scoring
        
    Returns:
        Tuple of (precision, recall, f1) lists
    """
    P, R, F1 = score(
        anonymized_texts, 
        original_texts, 
        lang="ru", 
        model_type=model_type,
        verbose=False
    )
    
    return P.numpy(), R.numpy(), F1.numpy()

def evaluate_anonymization_quality(test_texts):
    """
    Оценка качества анонимизации на наборе тестовых текстов
    
    Args:
        test_texts: Список тестовых текстов
        
    Returns:
        DataFrame с результатами оценки
    """
    results = []
    
    for i, text in enumerate(tqdm(test_texts, desc="Processing texts")):
        # Применяем оба метода анонимизации
        masked_text, _ = mask_anonymizer.extract_and_anonymize(text)
        replaced_text, _ = replace_anonymizer.extract_and_anonymize(text)
        
        # Вычисляем BERTScore для маскированного текста
        mask_precision, mask_recall, mask_f1 = calculate_bert_score(
            [text], [masked_text]
        )
        
        # Вычисляем BERTScore для замененного текста
        replace_precision, replace_recall, replace_f1 = calculate_bert_score(
            [text], [replaced_text]
        )
        
        # Сохраняем результаты
        result = {
            'text_id': i,
            'mask_precision': mask_precision[0],
            'mask_recall': mask_recall[0],
            'mask_f1': mask_f1[0],
            'replace_precision': replace_precision[0],
            'replace_recall': replace_recall[0],
            'replace_f1': replace_f1[0]
        }
        results.append(result)
    
    # Преобразуем в DataFrame
    df = pd.DataFrame(results)
    
    # Вычисляем статистику
    print("\n=== BERTScore Statistics ===")
    
    # Статистика для метода маскирования
    print("\nMask Method:")
    print(f"Mean Precision: {df['mask_precision'].mean():.4f}")
    print(f"Mean Recall: {df['mask_recall'].mean():.4f}")
    print(f"Mean F1: {df['mask_f1'].mean():.4f}")
    print(f"Median F1: {df['mask_f1'].median():.4f}")
    
    # Статистика для метода замены
    print("\nReplace Method:")
    print(f"Mean Precision: {df['replace_precision'].mean():.4f}")
    print(f"Mean Recall: {df['replace_recall'].mean():.4f}")
    print(f"Mean F1: {df['replace_f1'].mean():.4f}")
    print(f"Median F1: {df['replace_f1'].median():.4f}")
    
    # Сравнение
    print("\n=== Comparison ===")
    print(f"Mean F1 difference (Replace - Mask): {df['replace_f1'].mean() - df['mask_f1'].mean():.4f}")
    print(f"Median F1 difference (Replace - Mask): {df['replace_f1'].median() - df['mask_f1'].median():.4f}")
    
    # Подсчет, какой метод лучше для каждого текста
    replace_better_count = (df['replace_f1'] > df['mask_f1']).sum()
    mask_better_count = (df['mask_f1'] > df['replace_f1']).sum()
    equal_count = (df['replace_f1'] == df['mask_f1']).sum()
    
    print(f"\nTexts where Replace method is better: {replace_better_count} ({replace_better_count/len(df)*100:.1f}%)")
    print(f"Texts where Mask method is better: {mask_better_count} ({mask_better_count/len(df)*100:.1f}%)")
    print(f"Texts with equal F1 scores: {equal_count} ({equal_count/len(df)*100:.1f}%)")
    
    return df

In [ ]:
# Создаем набор тестовых текстов для оценки
test_texts = [
    "Иванов Иван Иванович, email: ivanov@example.com, карта: 1234 5678 9012 3456, ОГРНИП: 123456789012345, IP: 192.168.1.1, свидетельство о рождении: VII-АБ №123456, КЛАДР: 77 01 001 36",
    "Сотрудник Петров Сергей Владимирович предоставил документы: паспорт 4509 123456, ИНН 123456789012, email: petrov.sergey@company.ru, телефон: +7 (916) 123-45-67.",
    "Клиент Сидорова Анна Павловна, адрес: г. Москва, ул. Тверская, д. 1, кв. 100, телефон: 8 (495) 123-45-67, email: sidorova.a@mail.ru.",
    "ИП Кузнецов Дмитрий Алексеевич, ОГРНИП 987654321098765, ИНН 987654321098, расчетный счет в банке: 40802810000000012345, карта: 4532 1234 5678 9012.",
    "Заявитель: Новикова Елена Игоревна, паспорт серии 1234 номер 567890, выдан 01.01.2010 ОВД по району Хамовники г. Москвы, код подразделения: 770-001, адрес регистрации: 125009, г. Москва, ул. Тверская, д. 12, кв. 34."
]

# Оцениваем качество анонимизации
results_df = evaluate_anonymization_quality(test_texts)

print("\nРезультаты оценки качества анонимизации:")
display(results_df)

## Заключение

В этом ноутбуке мы представили полную систему анонимизации текста на русском языке, которая:

1. **Распознает различные типы персональных данных**:
   - Имена и фамилии с помощью библиотеки Natasha
   - Структурированные данные с помощью регулярных выражений

2. **Поддерживает два режима анонимизации**:
   - Маскирование (замена на символы #)
   - Замена (генерация похожих данных с сохранением структуры)

3. **Сохраняет морфологические характеристики** русского языка при замене имен и фамилий

4. **Валидирует форматы данных** перед анонимизацией (алгоритм Луна для карт, контрольные суммы для ИНН и ОГРНИП)

5. **Позволяет оценивать качество** анонимизации с помощью BERTScore

Система может быть использована для анонимизации документов, переписки, баз данных и других текстовых материалов, содержащих персональные данные.